# **Case Bolsista - Análise de Dados & Melhoria de Processos**

In [ ]:
import sqlite3 as sqlite
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 200)

**Importando base de dados**

In [ ]:
dfDados = pd.read_excel('base_de_dados.xlsx')

In [ ]:
# Convertendo tipo das colunas "nr_tat_dias_uteis" e "nr_pedidos" de float para integer

dfDados['nr_tat_dias_uteis'] = dfDados['nr_tat_dias_uteis'].astype('Int64')
dfDados['nr_pedidos'] = dfDados['nr_pedidos'].astype('Int64')

# Corrigindo tipo da coluna "dt_indicador"
dfDados['dt_indicador'] = dfDados['dt_indicador'].dt.date

In [ ]:
# Replace para corrigir nome REGI�O METROPOLITANA na coluna "des_capital_interior"

dfDados['des_capital_interior'] = dfDados['des_capital_interior'].str.replace('REGI�O METROPOLITANA', 'REGIAO METROPOLITANA')

In [ ]:
# Informações da base de dados

dfDados.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28170 entries, 0 to 28169
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   uf_sigla              28157 non-null  object
 1   nome_microrregiao     28157 non-null  object
 2   nome_municipio        28157 non-null  object
 3   des_capital_interior  28158 non-null  object
 4   des_canal             28010 non-null  object
 5   cod_marca             28170 non-null  object
 6   dt_indicador          28170 non-null  object
 7   st_entrega            28168 non-null  object
 8   nr_tat_dias_uteis     27639 non-null  Int64 
 9   nr_pedidos            27639 non-null  Int64 
dtypes: Int64(2), object(8)
memory usage: 2.2+ MB


**Conectando ao SQLite**

In [ ]:
conexao = sqlite.connect('banco_de_dados.db')

print("Conexão estabelecida!");

Conexão estabelecida!


**Salvando base de dados no banco SQLite**

In [ ]:
dfDados.to_sql('pedidos_logistica', conexao, if_exists='replace', index=False)

28170

## **Tratamento dos Dados**

**Removendo valores nulos**

In [ ]:
dfDados.isnull().sum()

uf_sigla                 13
nome_microrregiao        13
nome_municipio           13
des_capital_interior     12
des_canal               160
cod_marca                 0
dt_indicador              0
st_entrega                2
nr_tat_dias_uteis       531
nr_pedidos              531
dtype: int64

In [ ]:
# Removendo UFs nulas

dfDados.dropna(subset=['uf_sigla'], inplace=True)
dfDados.reset_index(drop=True, inplace=True)

In [ ]:
dfDados.isnull().sum()

uf_sigla                  0
nome_microrregiao         0
nome_municipio            0
des_capital_interior      0
des_canal               160
cod_marca                 0
dt_indicador              0
st_entrega                2
nr_tat_dias_uteis       531
nr_pedidos              531
dtype: int64

In [ ]:
# Removendo canais nulos

dfDados.dropna(subset=['des_canal'], inplace=True)
dfDados.reset_index(drop=True, inplace=True)

In [ ]:
dfDados.isnull().sum()

uf_sigla                  0
nome_microrregiao         0
nome_municipio            0
des_capital_interior      0
des_canal                 0
cod_marca                 0
dt_indicador              0
st_entrega                1
nr_tat_dias_uteis       530
nr_pedidos              530
dtype: int64

In [ ]:
# Removendo situações nulas

dfDados.dropna(subset=['st_entrega'], inplace=True)
dfDados.reset_index(drop=True, inplace=True)

In [ ]:
dfDados.isnull().sum()

uf_sigla                  0
nome_microrregiao         0
nome_municipio            0
des_capital_interior      0
des_canal                 0
cod_marca                 0
dt_indicador              0
st_entrega                0
nr_tat_dias_uteis       529
nr_pedidos              529
dtype: int64

Não foram removidos valores nulos de "nr_tat_dias_uteis" e "nr_pedidos", pois estas tuplas representam as situações Sinistro e Devolução. Sinistro e Devolução não tem TAT e número de pedidos.

In [ ]:
dfDados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27996 entries, 0 to 27995
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   uf_sigla              27996 non-null  object
 1   nome_microrregiao     27996 non-null  object
 2   nome_municipio        27996 non-null  object
 3   des_capital_interior  27996 non-null  object
 4   des_canal             27996 non-null  object
 5   cod_marca             27996 non-null  object
 6   dt_indicador          27996 non-null  object
 7   st_entrega            27996 non-null  object
 8   nr_tat_dias_uteis     27467 non-null  Int64 
 9   nr_pedidos            27467 non-null  Int64 
dtypes: Int64(2), object(8)
memory usage: 2.2+ MB


In [ ]:
dfDados.to_sql('pedidos_logistica', conexao, if_exists='replace', index=False)

27996

**Ajustes textuais na coluna "des_capital_interior"**

In [ ]:
print(dfDados.des_capital_interior.unique())

['INTERIOR' 'REGIAO METROPOLITANA' 'CAPITAL']


In [ ]:
query = '''UPDATE pedidos_logistica SET des_capital_interior = 'Interior' WHERE des_capital_interior = \'INTERIOR\'; '''
c = conexao.cursor()
c.execute(query)
conexao.commit()

query = '''UPDATE pedidos_logistica SET des_capital_interior = 'Região Metropolitana' WHERE des_capital_interior = \'REGIAO METROPOLITANA\'; '''
c = conexao.cursor()
c.execute(query)
conexao.commit()

query = '''UPDATE pedidos_logistica SET des_capital_interior = 'Capital' WHERE des_capital_interior = \'CAPITAL\'; '''
c = conexao.cursor()
c.execute(query)
conexao.commit()

query = '''SELECT * FROM pedidos_logistica'''
dfDados = pd.read_sql(query, conexao)

**Ajustes textuais na coluna "des_canal"**

In [ ]:
print(dfDados.des_canal.unique())

['VD' 'Loja' 'ECOMM']


In [ ]:
query = '''UPDATE pedidos_logistica SET des_canal = 'LOJA' WHERE des_canal = \'Loja\'; '''
c = conexao.cursor()
c.execute(query)
conexao.commit()

query = '''SELECT * FROM pedidos_logistica'''
dfDados = pd.read_sql(query, conexao)

**Ajustes textuais e unificando valores na coluna "st_entrega"**

In [ ]:
print(dfDados.st_entrega.unique())

['Prazo' 'Atraso' 'Sinistro' 'Devolucao' 'Entregue no Prazo'
 'Não Entregue no Prazo']


In [ ]:
query = '''UPDATE pedidos_logistica SET st_entrega = 'Devolução' WHERE st_entrega = \'Devolucao\'; '''
c = conexao.cursor()
c.execute(query)
conexao.commit()

In [ ]:
# Quantidade de dados para cada situação

print(dfDados.st_entrega.value_counts())

Entregue no Prazo        14353
Prazo                    11432
Atraso                    1677
Devolucao                  501
Sinistro                    28
Não Entregue no Prazo        5
Name: st_entrega, dtype: int64


In [ ]:
# Unificando valores das situações Prazo e Entregue no Prazo

query = '''UPDATE pedidos_logistica SET st_entrega = 'Entregue no Prazo' WHERE st_entrega = \'Prazo\'; '''

c = conexao.cursor()
c.execute(query)
conexao.commit()

query = '''SELECT * FROM pedidos_logistica'''
dfDados = pd.read_sql(query, conexao)

In [ ]:
# Quantidade de dados para cada situação pós unificação das situações

print(dfDados.st_entrega.value_counts())

Entregue no Prazo        25785
Atraso                    1677
Devolução                  501
Sinistro                    28
Não Entregue no Prazo        5
Name: st_entrega, dtype: int64


**Unificando valores da coluna "cod_marca"**

In [ ]:
print(dfDados.cod_marca.unique())

['EUD' 'EUD - Eudora' 'EUDORA' 'ECOM MONO' 'ECOMM MONO' 'Ecom Mono']


In [ ]:
# Quantidade de dados para cada marca

print(dfDados.cod_marca.value_counts())

ECOMM MONO      14298
EUD             13593
EUD - Eudora       40
Ecom Mono          36
ECOM MONO          24
EUDORA              5
Name: cod_marca, dtype: int64


In [ ]:
# Unificando valores para EUDORA

query = '''UPDATE pedidos_logistica SET cod_marca = 'EUDORA' 
           WHERE cod_marca = \'EUD\' OR cod_marca = \'EUD - Eudora\'; '''

c = conexao.cursor()
c.execute(query)
conexao.commit()

In [ ]:
# Unificando valores para ECOMM MONO

query = '''UPDATE pedidos_logistica SET cod_marca = 'ECOMM MONO' 
           WHERE cod_marca = \'ECOM MONO\' OR cod_marca = \'Ecom Mono\'; '''

c = conexao.cursor()
c.execute(query)
conexao.commit()

query = '''SELECT * FROM pedidos_logistica'''
dfDados = pd.read_sql(query, conexao)

In [ ]:
# Quantidade de dados para cada marca pós unificação das marcas

print(dfDados.cod_marca.value_counts())

ECOMM MONO    14358
EUDORA        13638
Name: cod_marca, dtype: int64


## **Dados**

In [ ]:
dfDic = pd.DataFrame(np.array([['uf_sigla', 'Sigla da UF', 'String', dfDados['uf_sigla'].str.len().max().astype(int), 'NOT NULL'], 
                            ['nome_microrregiao', 'Nome da microrregião', 'String', dfDados['nome_microrregiao'].str.len().max().astype(int), 'NOT NULL'], 
                            ['nome_municipio', 'Nome do município', 'String', dfDados['nome_municipio'].str.len().max().astype(int), 'NOT NULL'],
                            ['des_capital_interior', 'Tipo de região da entrega', 'String', dfDados['des_capital_interior'].str.len().max().astype(int), 'NOT NULL'],
                            ['des_canal', 'Canal de distribuição', 'String', dfDados['des_canal'].str.len().max().astype(int), 'NOT NULL'],
                            ['cod_marca', 'Marca', 'String', dfDados['cod_marca'].str.len().max().astype(int), 'NOT NULL'],
                            ['dt_indicador', 'Data da chegada do pedido dentro dos sistemas de Logística', 'Date', '', 'NOT NULL'],
                            ['st_entrega', 'Situação da entrega', 'String', dfDados['st_entrega'].str.len().max().astype(int), 'NOT NULL'],
                            ['nr_tat_dias_uteis', 'Tempo de atendimento total em dias úteis', 'Integer', '', ''],
                            ['nr_pedidos', 'Quantidade de pedidos da entrega', 'Integer', '', ''],
                  ]), 
                  columns=['Coluna', 'Descrição', 'Tipo', 'Tamanho', 'Restrições'])

dfDic.index = np.arange(1, 11)
dfDic[['Coluna', 'Tipo', 'Tamanho', 'Restrições', 'Descrição']]

,Coluna,Tipo,Tamanho,Restrições,Descrição
1,uf_sigla,String,2,NOT NULL,Sigla da UF
2,nome_microrregiao,String,36,NOT NULL,Nome da microrregião
3,nome_municipio,String,31,NOT NULL,Nome do município
4,des_capital_interior,String,20,NOT NULL,Tipo de região da entrega
5,des_canal,String,5,NOT NULL,Canal de distribuição
6,cod_marca,String,10,NOT NULL,Marca
7,dt_indicador,Date,,NOT NULL,Data da chegada do pedido dentro dos sistemas de Logística
8,st_entrega,String,21,NOT NULL,Situação da entrega
9,nr_tat_dias_uteis,Integer,,,Tempo de atendimento total em dias úteis
10,nr_pedidos,Integer,,,Quantidade de pedidos da entrega


In [ ]:
# Quantidade de valores nulos por coluna

dfDados.isnull().sum()

uf_sigla                  0
nome_microrregiao         0
nome_municipio            0
des_capital_interior      0
des_canal                 0
cod_marca                 0
dt_indicador              0
st_entrega                0
nr_tat_dias_uteis       529
nr_pedidos              529
dtype: int64

**Exportar dados em CSV**

In [ ]:
dfDados['nr_tat_dias_uteis'] = dfDados['nr_tat_dias_uteis'].astype('Int64')
dfDados['nr_pedidos'] = dfDados['nr_pedidos'].astype('Int64')

dfDados.to_csv('dados.csv', index=False, header=True, sep=';')

**Exportar dados em XLSX**

In [ ]:
dfDados.columns = ['UF', 'Microrregião', 'Município', 'Região', 'Canal de Distribuição', 'Marca', 'Data Entrada', 'Situação da Entrega', 'TAT em dias úteis', 'Quantidade de Pedidos']
dfDados.to_excel('dados.xlsx', index=False, header=True)